In [44]:
import pandas as pd
import math
import plotly.express as px

# Cleaning Sample History

In [2]:
df = pd.read_csv("Sample-History.csv")
df = df.rename(columns = {"Title": "History"})

In [3]:
df['Date'] = pd.to_datetime(df['Date'])

In [4]:
df['Day']= df['Date'].dt.day
df['Month']= df['Date'].dt.month
df['Year']= df['Date'].dt.year
df['Day_of_week'] = df['Date'].dt.dayofweek # monday = 0, sunday = 6

In [5]:
df

,History,Date,Day,Month,Year,Day_of_week
0,Better Call Saul: Season 5: Something Unforgiv...,2023-02-15,15,2,2023,2
1,Better Call Saul: Season 5: Bad Choice Road,2023-02-15,15,2,2023,2
2,Better Call Saul: Season 5: Bagman,2023-02-15,15,2,2023,2
3,Better Call Saul: Season 5: JMM,2023-02-15,15,2,2023,2
4,Better Call Saul: Season 5: Dedicado a Max,2023-02-14,14,2,2023,1
...,...,...,...,...,...,...
272,New Girl: Season 2: Neighbors,2021-11-21,21,11,2021,6
273,New Girl: Season 2: Fluffer,2021-11-21,21,11,2021,6
274,Squid Game: Season 1: One Lucky Day,2021-11-21,21,11,2021,6
275,Squid Game: Season 1: VIPS,2021-11-18,18,11,2021,3


In [6]:
df['Title'] = df['History'].str.rsplit(': ', 2).str[0]
df['Season'] = df['History'].str.rsplit(': ', 2).str[1]
df['Episode'] = df['History'].str.rsplit(': ', 2).str[2]

In [7]:
df['Type'] = df['Episode'].apply(lambda x : 'Movie' if (pd.isna(x)==True) else 'TV')

Right now, movies that have a colon in their title have an incomplete value in the `Title`. I correct that below:

In [8]:
tv = df[df['Type']!='Movie']
movies = df[df['Type']=='Movie']
movies['Title'] = movies['History']
movies['Season'] = None

<ipython-input-8-59af5807a4c7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['Title'] = movies['History']
<ipython-input-8-59af5807a4c7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['Season'] = None


In [9]:
tv.head()

,History,Date,Day,Month,Year,Day_of_week,Title,Season,Episode,Type
0,Better Call Saul: Season 5: Something Unforgiv...,2023-02-15,15,2,2023,2,Better Call Saul,Season 5,Something Unforgivable,TV
1,Better Call Saul: Season 5: Bad Choice Road,2023-02-15,15,2,2023,2,Better Call Saul,Season 5,Bad Choice Road,TV
2,Better Call Saul: Season 5: Bagman,2023-02-15,15,2,2023,2,Better Call Saul,Season 5,Bagman,TV
3,Better Call Saul: Season 5: JMM,2023-02-15,15,2,2023,2,Better Call Saul,Season 5,JMM,TV
4,Better Call Saul: Season 5: Dedicado a Max,2023-02-14,14,2,2023,1,Better Call Saul,Season 5,Dedicado a Max,TV


In [10]:
tv['Season'] = tv['Season'].str.split().str[1]
tv['Season'] = tv['Season'].astype(int)

<ipython-input-10-d2b08afe8aa9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv['Season'] = tv['Season'].str.split().str[1]


In [11]:
df = pd.concat([movies, tv], ignore_index = True)

Issues:

1. (**SOLVED**) TV shows that have a colon in their title need to be handled differently. See `Avatar: The Last Airbender`.
    - Should we consider only calling split() on ': Season', ': Series', ': Book', etc? An obvious issue is it would be hard to compile an exhaustive such list. Other examples include Volume, Chapter, etc.
    - Alternatively, we could keep a list of all TV shows from the Netflix dataset that do have a colon. If any show then appears in that list, we could then make sure its first colon is not counted in the above procedure. Again, an obvious issue is figuring out how to determine that a given show is in that list (since this has to occur before we do any cleaning)
    - A potential solution is to do split on colon occurences in reverse order. For instance, only split based on the last two occurences of the colon character, and leave earlier colon occurences alone.
    
2. Movie titles may not exactly match the Netflix Dataset. See `Major (Telugu)`, which appears in the Netflix dataset just as `Major`, but in 3 different rows with 3 different languages.
    - One solution would be to remove the `([language])` substring in the sample dataset, and drop duplicates in the Netflix dataset (since the only differences between the duplicates would be the language).
    - Another solution would be to instead fuzzy match based on string similarity. Theoretically, this should work since exact matches would just have a 100% score. Plus, if titles in the Netflix dataset have this issue too (i saw at least one occurence), this might be able to handle those edge cases. Based on past experience, can be pain to implement (that being said, the logic is simpler than what I've previously had to deal with).
        - We could also think about first doing exact string matching, and then performing fuzzy matching on the remaining titles that haven't been matched.
    - Another solution is to just ignore :) and hope it doesn't affect our recommendations much
3. TV Shows that have a colon in their episode name. See `New Girl: Season 6: Operation: Bobcat`.
    - Similar issue to 1, but flipped. Hard to solve both issues 1 and 3. Maybe just ignore?

# Merging with Netflix Data

In [12]:
titles = pd.read_csv("titles.csv")

In [34]:
merged = df.merge(titles, left_on = 'Title', right_on = 'title', how = 'inner')
# only 7 rows were dropped

In [38]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          270 non-null    datetime64[ns]
 1   Day           270 non-null    int64         
 2   Month         270 non-null    int64         
 3   Year          270 non-null    int64         
 4   Day_of_week   270 non-null    int64         
 5   Title         270 non-null    object        
 6   Season        268 non-null    object        
 7   Episode       268 non-null    object        
 8   Type          270 non-null    object        
 9   description   270 non-null    object        
 10  release_year  270 non-null    int64         
 11  runtime       270 non-null    int64         
 12  genres        270 non-null    object        
 13  imdb_score    270 non-null    float64       
 14  imdb_votes    270 non-null    float64       
 15  tmdb_score    270 non-null    float64   

In [36]:
cols_to_drop = ['type', 'production_countries', 'imdb_id', 'age_certification', 'id', 'title', 'seasons',
               'tmdb_popularity']
merged = merged.drop(cols_to_drop, axis = 1)

In [37]:
merged.head()

,Date,Day,Month,Year,Day_of_week,Title,Season,Episode,Type,description,release_year,runtime,genres,imdb_score,imdb_votes,tmdb_score
0,2022-04-08,8,4,2022,4,Delhi Belly,None,NaN,Movie,"Three unsuspecting, average guys find themselv...",2011,102,"['comedy', 'action', 'crime', 'thriller']",7.5,29927.0,7.0
1,2021-11-24,24,11,2021,2,Most Eligible Bachelor,None,NaN,Movie,Akhil and Pooja have a love story with a touch...,2021,170,"['drama', 'romance']",6.1,3261.0,5.7
2,2023-02-15,15,2,2023,2,Better Call Saul,5,Something Unforgivable,TV,Six years before Saul Goodman meets Walter Whi...,2015,49,"['drama', 'crime']",8.8,438575.0,8.5
3,2023-02-15,15,2,2023,2,Better Call Saul,5,Bad Choice Road,TV,Six years before Saul Goodman meets Walter Whi...,2015,49,"['drama', 'crime']",8.8,438575.0,8.5
4,2023-02-15,15,2,2023,2,Better Call Saul,5,Bagman,TV,Six years before Saul Goodman meets Walter Whi...,2015,49,"['drama', 'crime']",8.8,438575.0,8.5


# Aggregation

Stats/Visualizations to create:
- Individual
    - Viewing history by day of week
    - Top 10 Shows Watched
    - Movies most rewatched
    - Viewing history by month
    - Viewing history trend over time
    - Average watch time per day
- Common
    - Most common TV shows
    - Most common movies
    - Most common genres
    - Favorite actors/directors
    - Match score
- Overall
    - Minutes spent vs all other users
    - Percentile for top shows

**For all of these, we could use number of minutes watched (using the runtime column) rather than just the number of episodes**

1. Total Minutes Watched

In [51]:
df = merged.groupby('Type').sum()

In [68]:
fig = px.pie(merged, values='runtime', names='Type', hole=.5)
fig.update_traces(hovertemplate='Total Watch Time: %{value} mins')
fig.show()

2. Most watched TV shows

In [76]:
df = merged[merged['Type']=='TV']
df = df.groupby('Title').sum()
df = df.reset_index()

In [80]:
df = df.sort_values(by='runtime', ascending= False)
# extract 10 most similar shows
df = df.head(10)

In [82]:
fig = px.bar(data_frame= df,
             x= "runtime",
             y= "Title",
             # text= "num_shared_actors",
             labels= {"runtime": "Minutes Watched"},
             text_auto= True)
fig.update_traces(hovertemplate='Total Watch Time: %{x} mins')
fig.show()

3. Most watched genre

In [135]:
df = merged
df['genres'] = df['genres'].apply(lambda x: x.replace('\'', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace('[', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace(']', ''))
df['genres'] = df['genres'].apply(lambda x: x.replace(' ', ''))

In [136]:
df = df['genres'].str.split(',', expand = True)

In [137]:
df = pd.DataFrame(df.apply(pd.Series.value_counts).sum(axis = 1)).reset_index()

In [140]:
fig = px.pie(df, values=0, names='index', hole=.5)
fig.update_traces(hovertemplate='Number of Titles Watched: %{value}')
fig.show()